## Order values minimums are not getting close to zero
Investigate why

In [1]:
from particletracking import dataframes, statistics

/home/ppxjd3/.local/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [14]:
with dataframes.DataStore("/media/data/Data/FirstOrder/PhaseDiagram/DimpledPlateFeb2021/1600.hdf5") as data:
    df = data.df

In [15]:
df

,x,y,r,Duty,order_r,order_i,neighbors,order,density,shape_factor,on_edge,order_r_long,order_i_long,neighbors_long,order_long
frame,,,,,,,,,,,,,,,
0,1013,591,18,700.0,0.000000e+00,0.000000,0,0.000000,0.504120,1.145543,False,-0.236008,-0.432361,6,0.000000
0,1505,857,18,700.0,3.269737e-01,0.050575,3,0.330862,0.671392,1.159897,False,0.442276,-0.097347,6,0.330862
0,951,642,18,700.0,2.286157e-01,-0.973517,1,1.000000,0.404068,1.164192,False,0.084545,-0.066865,7,1.000000
0,1662,307,18,700.0,2.220446e-16,-0.936996,2,0.936996,0.488093,1.153936,False,0.376304,-0.467314,6,0.936996
0,653,1530,18,700.0,0.000000e+00,0.000000,0,0.000000,0.413688,1.223241,False,0.313464,-0.442336,5,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50519,1921,465,18,500.0,8.067018e-01,-0.590959,1,1.000000,0.907101,1.323396,True,0.877957,0.013457,4,1.000000
50519,226,1355,18,500.0,0.000000e+00,0.000000,0,0.000000,0.902280,1.300518,True,0.975058,0.109512,4,0.000000
50519,1739,142,18,500.0,5.697650e-01,-0.821808,1,1.000000,0.804899,1.266122,True,0.846150,-0.202044,4,1.000000


In [20]:
f = df.loc[45000]

In [17]:
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [63]:
sp.delaunay_plot_2d(sp.Delaunay(f[['x', 'y']].values))
plt.scatter(f.x, f.y, c=f.order_long)
plt.plot(f.x[f.neighbors==0], f.y[f.neighbors==0], 'x')

# plt.colorbar()

In [22]:
plt.hist(f.order[f.neighbors==2], bins=100)

(array([ 1.,  0.,  1.,  2.,  0.,  2.,  1.,  2.,  0.,  1.,  2.,  0.,  3.,
         1.,  2.,  0.,  3.,  2.,  1.,  2.,  2.,  1.,  3.,  3.,  1.,  1.,
         1.,  1.,  2.,  1.,  2.,  0.,  1.,  1.,  0.,  2.,  1.,  2.,  2.,
         2.,  2.,  5.,  3.,  4.,  0.,  2.,  1.,  1.,  0.,  1.,  5.,  3.,
         5.,  1.,  1.,  2.,  5.,  1.,  3.,  0.,  2.,  1.,  1.,  4.,  8.,
         2.,  2.,  3.,  5.,  4.,  3.,  4.,  1.,  2.,  7.,  4.,  2.,  5.,
         3.,  6.,  8.,  6.,  4.,  7.,  5.,  7.,  6.,  7., 10., 11.,  9.,
        12., 10.,  6., 11., 12., 17., 19., 18., 78.]),
 array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
        0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
        0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
        0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
        0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
        0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.

In [26]:
f.loc[f.neighbors==1]['order']

frame
30000    1.0
30000    1.0
30000    1.0
30000    1.0
30000    1.0
        ... 
30000    1.0
30000    1.0
30000    1.0
30000    1.0
30000    1.0
Name: order, Length: 340, dtype: float32

In [22]:
import numpy as np
import scipy.spatial as sp


def order_process(features, threshold=2.3):
    points = features[['x', 'y', 'r']].values
    threshold *= np.mean(points[:, 2])
    orders, neighbors = order_and_neighbors(points[:, :2], threshold)
    features['order_r'] = np.real(orders).astype('float32')
    features['order_i'] = np.imag(orders).astype('float32')
    features['neighbors'] = neighbors
    return features


def order_process_mean(features, threshold=2.3):
    points = features[['x_mean', 'y_mean', 'r']].values
    threshold *= np.mean(points[:, 2])
    orders, neighbors = order_and_neighbors(points[:, :2], threshold)
    features['order_r_mean'] = np.real(orders).astype('float32')
    features['order_i_mean'] = np.imag(orders).astype('float32')
    features['neighbors_mean'] = neighbors
    return features


def order_and_neighbors(points, threshold):
    list_indices, point_indices = find_delaunay_indices(points)
    vectors = find_vectors(points, list_indices, point_indices)
    filtered = filter_vectors(vectors, threshold)
    angles = calculate_angles(vectors)
    orders, neighbors = calculate_orders(angles, list_indices, filtered)
    neighbors = np.real(neighbors).astype('uint8')
    return orders, neighbors


def find_delaunay_indices(points):
    tess = sp.Delaunay(points)
    return tess.vertex_neighbor_vertices


def find_vectors(points, list_indices, point_indices):
    repeat = list_indices[1:] - list_indices[:-1]
    return points[point_indices] - np.repeat(points, repeat, axis=0)


def filter_vectors(vectors, threshold):
    length = np.linalg.norm(vectors, axis=1)
    return length < threshold


def calculate_angles(vectors):
    angles = np.angle(vectors[:, 0] + 1j * vectors[:, 1])
    return angles


def calculate_orders(angles, list_indices, filtered):
    # calculate summand for every angle
    step = np.exp(6j * angles)
    # set summand to zero if bond length > threshold
    step *= filtered
    list_indices -= 1
    # sum the angles and count neighbours for each particle
    stacked = np.cumsum((step, filtered), axis=1)[:, list_indices[1:]]
    stacked[:, 1:] = np.diff(stacked, axis=1)
    neighbors = stacked[1, :]
    indxs = neighbors != 0
    orders = np.zeros_like(neighbors)
    orders[indxs] = stacked[0, indxs] / neighbors[indxs]
    return orders, neighbors

In [23]:
points = f[['x', 'y', 'r']]

In [24]:
threshold = 4 * np.mean(points.r)

In [25]:
points = points.values

In [26]:
list_indices, point_indices = find_delaunay_indices(points)

In [49]:
vectors = find_vectors(points, list_indices, point_indices)

In [51]:
vectors.shape

(15068, 3)

In [53]:
list_indices

array([    0,     7,    15, ..., 15055, 15061, 15068], dtype=int32)

In [58]:
x_repeat = np.repeat(points[:, 0], list_indices[1:]-list_indices[:-1])
y_repeat = np.repeat(points[:, 1], list_indices[1:]-list_indices[:-1])

In [62]:
plt.quiver(x_repeat, y_repeat, vectors[:, 0], vectors[:, 1], angles='xy', scale_units='xy', scale=1)

In [28]:
filtered = filter_vectors(vectors, threshold)

In [29]:
angles = calculate_angles(vectors)
angles

array([-3.09814176, -3.13444992,  2.07363954, ...,  3.14159265,
        3.1371679 , -2.6845239 ])

In [38]:
vectors, np.linalg.norm(vectors, axis=1)

(array([[ -46,   -2,    0],
        [-140,   -1,   -1],
        [ -22,   40,    0],
        ...,
        [ -85,    0,    0],
        [-226,    1,    0],
        [ -61,  -30,    0]], dtype=int32),
 array([ 46.04345773, 140.00714267,  45.65084884, ...,  85.        ,
        226.00221238,  67.9779376 ]))

In [39]:
filtered

array([ True, False,  True, ..., False, False,  True])

In [42]:
step = np.exp(6j*angles)
step.tolist()

[(0.7464321210857621-0.6654615605813834j),
 (0.36768059815398774+0.9299521373388664j),
 (0.8892573236242688+0.4574072718930062j),
 (0.9473889049087971+0.3200847744828081j),
 (1-7.347880794884119e-16j),
 (0.3552599271776968-0.934767556209402j),
 (0.8835781746679898-0.46828368458700675j),
 (0.3658191981392423-0.9306859375067197j),
 (0.771717075018483-0.6359660023341791j),
 (-0.5444054465237359-0.838822215845105j),
 (0.36152315200919416+0.9323631323477656j),
 (-0.9893154138790061-0.14579098690046252j),
 (0.42868339800560884-0.9034547826395989j),
 (-0.3268712171142462-0.9450688903049614j),
 (0.20846306717032947-0.9780302396275581j),
 (0.9329029346609201-0.360127914081987j),
 (0.9127194015845954-0.4085869478716349j),
 (0.5766833357647252+0.8169677657357537j),
 (0.352-0.936j),
 (0.1007531040097706+0.9949114593934459j),
 (-0.752192+0.658944j),
 (-0.2505122194222281-0.9681134375268993j),
 (0.3768775603095131+0.9262630860263996j),
 (0.6719450354575441+0.7406010189865795j),
 (-0.4427905989508108

In [43]:
step *= filtered
step.tolist()

[0j,
 0j,
 0j,
 0j,
 0j,
 0j,
 0j,
 0j,
 0j,
 -0j,
 0j,
 (-0.9893154138790061-0.14579098690046252j),
 0j,
 -0j,
 (0.20846306717032947-0.9780302396275581j),
 0j,
 0j,
 0j,
 (0.352-0.936j),
 0j,
 (-0.752192+0.658944j),
 -0j,
 (0.3768775603095131+0.9262630860263996j),
 0j,
 (-0+0j),
 0j,
 (-0+0j),
 0j,
 -0j,
 -0j,
 -0j,
 0j,
 (-0+0j),
 0j,
 -0j,
 0j,
 -0j,
 0j,
 0j,
 0j,
 0j,
 0j,
 0j,
 0j,
 0j,
 0j,
 0j,
 (0.8067018526898462+0.5909586456485509j),
 0j,
 0j,
 (-0.957540097938604-0.28830012285763745j),
 0j,
 (-0+0j),
 -0j,
 (-0.9554102287890078-0.2952817209468538j),
 (1-7.347880794884119e-16j),
 (-0.9949114593934459+0.10075310400977011j),
 0j,
 (-0.752192-0.658944j),
 0j,
 (-0.9893154138790062-0.1457909869004615j),
 (-0+0j),
 -0j,
 (-0.9996431497496586-0.026712790168411972j),
 0j,
 0j,
 (-0+0j),
 0j,
 -0j,
 (-0+0j),
 0j,
 0j,
 0j,
 0j,
 0j,
 -0j,
 (0.4014861429155372-0.9158650976245382j),
 (0.5011888141865171+0.8653379527874141j),
 0j,
 0j,
 0j,
 -0j,
 0j,
 0j,
 0j,
 (-0+0j),
 0j,
 0j,
 -0j

In [45]:
list_indices -= 1

In [53]:
stacked = np.cumsum((step, filtered), axis=1)[:, list_indices[1:]]
stacked

array([[ 0.00000000e+00  +0.j        , -7.80852347e-01  -1.12382123j,
        -8.04166786e-01  -0.47461414j, ...,
        -2.16602953e+03-347.72073996j, -2.16449600e+03-348.87458839j,
        -2.16382677e+03-348.13153239j],
       [ 0.00000000e+00  +0.j        ,  2.00000000e+00  +0.j        ,
         5.00000000e+00  +0.j        , ...,
         5.67700000e+03  +0.j        ,  5.67900000e+03  +0.j        ,
         5.68000000e+03  +0.j        ]])

In [55]:
stacked[:, 1:] = np.diff(stacked, axis=1)
stacked

array([[ 0.        +0.j        , -0.78085235-1.12382123j,
        -0.02331444+0.64920709j, ...,  1.277108  +1.48414856j,
         1.53352749-1.15384843j,  0.66922924+0.743056j  ],
       [ 0.        +0.j        ,  2.        +0.j        ,
         3.        +0.j        , ...,  2.        +0.j        ,
         2.        +0.j        ,  1.        +0.j        ]])

In [57]:
neighbors = stacked[1, :]
neighbors.real

array([0., 2., 3., ..., 2., 2., 1.])

In [59]:
stacked.shape

(2, 1994)

In [61]:
indxs = neighbors != 0

In [62]:
indxs

array([False,  True,  True, ...,  True,  True,  True])

In [63]:
points.shape

(1994, 3)

In [64]:
list_indices[1:]

array([    6,    14,    23, ..., 18141, 18156, 18169], dtype=int32)

In [71]:

stacked_temp = np.cumsum((step, filtered), axis=1)

In [76]:
stacked_temp[:, 1:] = np.diff(stacked_temp, axis=1)


In [81]:
neighbors = stacked[1, :]
neighbors

array([0.+0.j, 2.+0.j, 3.+0.j, ..., 2.+0.j, 2.+0.j, 1.+0.j])

In [82]:
indxs = neighbors != 0

In [84]:
indxs

array([False,  True,  True, ...,  True,  True,  True])

In [88]:
orders = np.zeros_like(neighbors)
orders[indxs] = stacked[0, indxs] / neighbors[indxs]

In [93]:
sorter = np.argsort(orders)

In [96]:
plt.plot(orders[sorter])
plt.plot(neighbors[sorter])

/home/ppxjd3/miniconda3/envs/ExperimentsNew/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/home/ppxjd3/miniconda3/envs/ExperimentsNew/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [98]:
orders_r = np.real(orders).astype('float32')
orders_i = np.imag(orders).astype('float32')

In [99]:
orders_mag = np.abs(orders_r + 1j*orders_i)

In [102]:
plt.plot(orders_mag[sorter])
plt.plot(neighbors[sorter])

/home/ppxjd3/miniconda3/envs/ExperimentsNew/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [103]:
x = np.linspace(0, 2*np.pi, 100)

In [104]:
y = np.exp(6j*x)

In [108]:
plt.plot(x, np.abs(y))

In [110]:
plt.plot(x, y**2)

/home/ppxjd3/miniconda3/envs/ExperimentsNew/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [111]:
def order_on_angles(a):
    steps = np.exp(6j*a)
    total = np.sum(steps)
    return total / len(a)

In [114]:
a = np.linspace(0, 2*np.pi, 6, endpoint=False)
a

array([0.        , 1.04719755, 2.0943951 , 3.14159265, 4.1887902 ,
       5.23598776])

In [121]:
b = np.random.rand(6)
b

array([0.81791106, 0.99330568, 0.84065648, 0.37599359, 0.79897802,
       0.85033799])

In [132]:
n = np.arange(1, 20, 1)
o = [order_on_angles(a + b/ni) for ni in n]

In [133]:
plt.plot(n, np.abs(o))

In [130]:
o

[(0.21599496420912584-0.5652886170277639j)]

In [135]:
order = f.order * f.neighbors / 6

In [138]:
plt.hist(order, bins=100)

(array([144.,   4.,   5.,   4.,   7.,   7.,   5.,   8.,   5.,   7.,   5.,
          5.,   7.,  11.,  12.,  10., 353.,  16.,  17.,  12.,   9.,  23.,
         15.,  17.,  22.,  17.,  27.,  29.,  28.,  35.,  37.,  39.,  62.,
         93.,  14.,  12.,   7.,  12.,  16.,   8.,  13.,  13.,  22.,  10.,
         20.,  22.,  20.,  23.,  24.,  23.,  10.,   5.,   4.,   7.,   4.,
          9.,   7.,  10.,  13.,  13.,   9.,  21.,  24.,  35.,  33.,  35.,
         27.,   2.,   3.,   3.,   3.,   2.,   2.,   9.,   5.,   7.,  10.,
         19.,  29.,  23.,  31.,  40.,  42.,  32.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   1.,   0.,   3.,   5.,   9.,  11.,  15.,  30.,
         36.]),
 array([0.        , 0.00996605, 0.01993209, 0.02989814, 0.03986419,
        0.04983024, 0.05979628, 0.06976233, 0.07972838, 0.08969443,
        0.09966047, 0.10962652, 0.11959257, 0.12955862, 0.13952467,
        0.14949071, 0.15945676, 0.1694228 , 0.17938885, 0.1893549 ,
        0.19932094, 0.209287  , 0.21925305, 0.

In [139]:
order = order_process(points)

/home/ppxjd3/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [140]:
order = order_process(f[['x', 'y', 'r']])

/home/ppxjd3/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ppxjd3/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ppxjd3/.local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [149]:
order_long = order_process(f[['x', 'y', 'r']], 5)

/home/ppxjd3/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ppxjd3/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ppxjd3/.local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [150]:
order['order'] = np.abs(order['order_r']+1j*order['order_i'])
order_long['order'] = np.abs(order_long['order_r']+1j*order_long['order_i'])

/home/ppxjd3/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ppxjd3/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [151]:
plt.hist(order.order, bins=100, alpha=0.5)
plt.hist(order_long.order, bins=100, alpha=0.5)

(array([ 1.,  1.,  2.,  2.,  1.,  0.,  4.,  2.,  1.,  3.,  6.,  5.,  2.,
         2.,  6.,  4.,  2.,  8.,  5.,  9.,  6.,  6.,  5.,  7.,  7.,  8.,
         8., 10., 10.,  8., 13., 11., 10., 12.,  9., 13., 12., 12., 13.,
        11.,  8., 10., 12.,  8., 12., 10., 19., 15.,  9., 17.,  8., 20.,
        19., 20., 10., 11., 18., 14., 15., 16., 16., 14., 16., 13., 12.,
        15., 18., 15., 16., 27., 20., 23., 20., 20., 18., 23., 32., 24.,
        24., 25., 25., 34., 22., 26., 29., 39., 26., 40., 43., 52., 41.,
        52., 64., 74., 73., 87., 91., 85., 99., 73.]),
 array([0.02095348, 0.03072642, 0.04049936, 0.05027229, 0.06004523,
        0.06981816, 0.0795911 , 0.08936403, 0.09913697, 0.1089099 ,
        0.11868284, 0.12845577, 0.13822871, 0.14800164, 0.15777458,
        0.16754751, 0.17732045, 0.18709339, 0.19686632, 0.20663926,
        0.21641219, 0.22618513, 0.23595807, 0.245731  , 0.25550392,
        0.26527688, 0.2750498 , 0.28482273, 0.2945957 , 0.30436862,
        0.31414154, 0.3239

In [152]:
order_long

,x,y,r,order_r,order_i,neighbors,order
frame,,,,,,,
30000,430,962,18,0.803653,-0.215170,6,0.831959
30000,516,203,18,0.117146,-0.515233,7,0.528383
30000,846,1130,18,0.277017,0.329355,6,0.430364
30000,1866,672,18,0.042602,-0.026302,7,0.050068
30000,786,1294,18,0.852400,0.084321,6,0.856561
...,...,...,...,...,...,...,...
30000,870,1925,17,0.755008,0.060417,4,0.757421
30000,1664,348,17,-0.878931,-0.333487,6,0.940071
30000,459,1760,18,0.848460,0.372358,5,0.926572


In [154]:
plt.subplot(1, 2, 1)
plt.scatter(order_long.x, order_long.y, c=order_long.order)
plt.subplot(1, 2, 2)
plt.scatter(order.x, order.y, c=order.order)

In [155]:
cutoffs = [2, 3, 4, 5, 6, 7, 8, 9, 10]

In [156]:
orders = [order_process(f[['x', 'y', 'r']], c) for c in cutoffs]

/home/ppxjd3/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ppxjd3/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ppxjd3/.local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [161]:
for c, o in zip(cutoffs, orders):
    o.order = np.abs(o.order_r + 1j*o.order_i)
    plt.hist(o.order, bins=100, histtype='step', label=c)
    plt.legend()

In [64]:
list_indices

array([    0,     7,    15, ..., 15055, 15061, 15068], dtype=int32)

In [66]:
repeat = list_indices[1:] - list_indices[:-1]
repeat

array([7, 8, 7, ..., 7, 6, 7], dtype=int32)

In [82]:
a = points[point_indices][:7]

In [83]:
end_points = np.repeat(points, repeat, axis=0)
b = end_points[:7]

In [97]:
plt.plot(a[:, 0], a[:, 1], '.')
plt.plot(b[:, 0], b[:, 1], '.')
plt.quiver(b[:, 0], b[:, 1], vectors[:, 0], vectors[:, 1], scale_units='xy', scale=1, angles='xy')
cir = plt.Circle((b[0, 0], b[0, 1]), 72, fill=False)
plt.gca().add_patch(cir)

In [89]:
vectors = a - b